## K-means clustering 실습 2

### 신용카드 사용 데이터


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

In [ ]:
# 데이터 불러오기: 고객 데이터셋

file_url = 'https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/customer.csv'
customer =

customer.info()

In [ ]:
customer.

# cc_num 신용카드 번호, category 카드 사용 분류, amt 사용 금액

In [ ]:


# unique 한 cc_num 개수는 100개

In [ ]:


# unique 한 category 개수는 11개

### 전처리 : Feature Engineering (피처 엔지니어링)


In [ ]:
# get_dummies 를 사용하여 category 카드 사용 분류 컬럼을 one-hot encoding

customer_dummy =
customer_dummy.head()

# category 를 모두 분류하고 true 라고 표시된 것은 해당 카드 사용 분류를 사용한 것이고, false 라고 표시된 것은 해당 카드 사용 분류를 사용하지 않은 것
# category 가 11개 이므로 더미 변수 11개가 추가됨

In [ ]:
# cc_num 을 제외하고, 나머지 amt 와 category 컬럼들을 곱하여 사용 금액을 계산

cat_list =

# amt(금액)로 변수 업데이트
for i in cat_list:
    customer_dummy[i] =

customer_dummy

In [ ]:
# 각 거래 건으로 정리된 데이터를 고객 cc_num 을 기준으로 groupby 하여 고객별 총 사용 금액 및 카테고리별 사용 금액 계산

customer_agg =

customer_agg.head()

In [ ]:
# customer_agg의 데이터 정보 보기


In [ ]:
# K-means 알고리즘은 거리(distance) 기반의 알고리즘이므로, 데이터의 스케일이 다르면 제대로된 군집화가 어려울 수 있음
# 따라서, 스케일링을 통해 데이터의 스케일을 맞추어 주어야 함
# 여기서는 StandardScaler 를 사용하여 각 변수의 평균이 0, 분산이 1이 되도록 스케일링

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_df = pd.DataFrame(,
            columns = ,
            index=) # 스케일링 후 데이터프레임으로 변환

In [ ]:
scaled_df.head()

# 다른 고객들과 비슷한 수준이면, 즉 평균에 가까울 경우는 0에 근접한 값을 나타내게 되고,
# 더 많이 사용했으면 더 큰 양수를, 더 적게 사용했으면 더 작은 음수값을 나타냄

### 엘보우 방법을 통한 적절한 k 값 찾기

In [ ]:
# 현재로써는 적절한 k 값을 모르기 때문에 엘보우 방법을 사용하여 적절한 k 값을 찾아보기
distance = []

for k in range(2, 10):
    k_model = KMeans(n_clusters=k)
    k_model.fit(scaled_df)
    labels = k_model.predict(scaled_df)
    distance.append(k_model.inertia_)

sns.lineplot(x=range(2,10), y=distance)

### 실루엣 계수 (silhouette score)

실루엣 계수는 클러스터 내부에서의 평균 거리(a)와, 최근접한 다른 클러스터 데이터와의 평균 거리(b)를 계산에 반영

실루엣 계수 = (b-a) / max(a,b)

In [ ]:
# 엘보우 기법과 같이 최적의 클러스터 수를 찾기 위해 실루엣 점수를 계산
# 엘보우 기법에서 적절한 클러스터 수를 찾지 못했을 때 대안으로 사용할 수 있음

from sklearn.metrics import silhouette_score

In [ ]:
silhouette = []

for k in range(2,10):
    k_model = KMeans(n_clusters=k)
    k_model.fit(scaled_df)
    labels = k_model.predict(scaled_df)
    silhouette.append(silhouette_score(scaled_df, labels))

In [ ]:
sns.lineplot(x=range(2,10), y=silhouette)

# 실루엣 점수가 가장 높은 k 값을 선택 -> k=4

### 최종 예측 모델 및 결과 해석


In [ ]:
k_model =

k_model.

labels =

In [ ]:
# labels 를 기존의 데이터프레임 scaled_df 에 새로운 컬럼으로 추가


In [ ]:
# 클러스터링 결과를 해석하기 위하여 label 별로 데이터 요약: label 별 평균값
scaled_df_mean =

# label 별 고객 수를 scaled_df_count 에 저장
scaled_df_count =

In [ ]:
# 'count' 로 이름 변경
scaled_df_count =

In [ ]:
# 데이터 합치기
scaled_df_all =

In [ ]:
scaled_df_all

### 해설

- 클러스터 0(label 0인 그룹)은 전체 지출액부터 각 범주의 금액까지 모두 타 클러스터에 비해 낮음

    - 여기에 속하는 고객 수는 45명으로 가장 많음

- 클러스터 1은 gas_transport 에서 상대적으로 조금 높은 지출을 보이지만, 전체 지출 및 각 범주에서 대부분 0 에 가까운 숫자를 보임

    - 여기에 속하는 고객 수는 37명으로 딱히 특징이 없는(?) 일반 고객군으로 분류

- 클러스터 2는 전체 지출금액부터 대다수의 범주에서 높은 지출을 나타냄

    - 여기에 속하는 고객 수는 10명으로 상위 10%의 VIP 고객에 해당함

- 클러스터 3은 home, travel 에서 다른 클러스터보다 지출이 높음
    
    - travel 이 매우 높게 나온 클러스터로, 여행에 관심이 많은 고객군으로 정의할 수 있음